# Word Substitutions

read the available WORDS corresponding to PSL video FILES</br>
seperate the NORMAL, AMBIGUOUS, & COMPOUND words

### dictionaries

##### supported_words2file
sw2f['normal']              = 'filename1'</br>
sw2f['ambiguous(meaning1)'] = 'filename2'</br>
sw2f['ambiguous(meaning2)'] = 'filename3'</br>
sw2f['compound-word']       = 'filename4'

##### supported_words2embedding
sw2e['normal']    = [0.5, 0.3, 0.1, 0.9, ... ]</br>
sw2e['ambiguous'] = [0.4, 0.6, 0.2, 0.1, ... ]</br>
?? sw2e['subword'] = [0.4, 0.6, 0.2, 0.1, ... ] ??

##### ambiguous2resolved
a2r['ambiguous1'] = ['a1(m1)', 'a1(m2)', 'a1(m3)', ...]</br>
a2r['ambiguous2'] = ['a2(m1)', 'a2(m2)', 'a2(m3)', ...]

##### first_subword2compound
fsw2c['subw1'] = ['sw1-sw2-sw3', 'sw1-sw2', 'sw1-sw2', ...]</br>
fsw2c['subw2'] = ['sw2-sw4-sw3', 'sw2-sw1', 'sw2-sw3', ...]

##### word2substitutes (substitutes response)
w2s['word'] = ['sub1', 'sub2', 'sub3', ...]

### mapping Procedure
split sentence</br>
for each word</br>
clean it from punctuation</br>
check if normal --> sw2f</br>
check if known ambiguous --> a2r </br>
--T-> w2s['word'] . extend( a2r['word'] )</br>
--F-> pass</br>
check if compound --> fsw2c</br>
--T-> for comp in comps, 
for subword in comp, 
if sentence words match till end, 
w2s[comp.replace('-','')] . append( comp )</br>
--F-> pass</br>
Match with sw2e</br>
--> w2s['word'].extend( a2r[most_similar] )

### dict preparation
read file2lang line by line</br>
for row.split('\t,\t')</br>
file = row[0]</br>
words = row[1:]</br>
for word in words:</br>
    sw2f[word] = file</br>
    if '(' in word:</br>
        temp = word.split('(')[0]</br>
        a2r[temp].append(word)</br>
        
    (el)if '-' in word:  # aword(mean-ing)
        temp= word.split('-')[0]
        fsw2c[temp].append(word)

    try
        sw2e_1[word] = lang_embeddings[word]
    except
        try
            sw2e_2[word] = lang_embeddings[word.replace(' ','')], ('-',''), (' ','-')
                .
                    .  
                        .
                            .
                        except
                            nomatch.append(word)
 

In [11]:
import numpy as np #read embd, dict
import os
def read_embeddings(path, encoding):
    dic={}
    with open(path, encoding=encoding) as f: #
        rows=  int(f.readline().split(' ')[0])

        for i in range(rows):
            line = f.readline()
            row = line.strip().split(' ')
            dic[row[0]] = np.array(row[1:], dtype=np.float32)  
                
    return dic

In [13]:
ur_embd_file = 'W2V300dim5winBulk.txt'
ur_embeddings = read_embeddings(os.path.join('txt',ur_embd_file), 'utf-8')
en_embd_file = 'English_W2V_100dm.txt'
en_embeddings = read_embeddings(os.path.join('txt',en_embd_file),'ISO-8859-1')
len(en_embeddings),len(en_embeddings)

(461449, 4027169)

In [118]:
ur_words = 'file2urdu.txt'
ur_sw2f  = {}
ur_sw2e  = {}
ur_sw2e_other  = {}
ur_a2r   = {}
ur_fsw2c = {}

en_words = 'file2english.txt'
en_sw2f  = {}
en_sw2e  = {}
en_sw2e_other  = {}
en_a2r   = {}
en_fsw2c = {}


found_space,removed_dash,replaced_dash_with_space,nothing_worked=[],[],[],[]
def put_embedding_in_dict(key, dict1,dict2, embd):
    try:
        dict1[key] = embd[key]
    except:
        try:
            t1 = key.replace(' ','')
            dict2[t1] = embd[t1]
            found_space.append(f'{key}___>{t1}')
        except:
            try:
                t2 = key.replace('-','')
                dict2[t2] = embd[t2]
                removed_dash.append(f'{key}___>{t2}')
            except:
                try:
                    t3=key.replace(' ','-')
                    dict2[t3] = embd[t3]
                    found_space.append(f'{key}___>{t3}')
                except:
                    try:
                        t4 = key.replace('-',' ')
                        dict2[t4] = embd[t4]
                        replaced_dash_with_space.append(f'{key}___>{t4}')
                    except:
                        nothing_worked.append(key)

def append_to_dict(dict, key, value):
    try:
        dict[key].append(value)
    except:
        dict[key] = [value]


def read_words_file(path,sw2f,sw2e,sw2e_other,embeddings,a2r,fsw2c,encoding='utf-8'):
    all_supported_words =[]
    with open(path, encoding=encoding)as f:
        lines = f.read().split("\n")
        for line in lines:
            toks = line.split('\t,\t')
            file = toks[0]
            words = toks[1:]
            for word in words:
                if word=='':
                    continue
                
                if '.' in word or ',' in word or '?' in word or '!' in word or '۔' in word or '؟' in word or ' ' in word or '،' in word:
                    print(f'---{word}---')

                sw2f[word]=file
                all_supported_words.append(word)

                if '(' in word:
                    a_word = word.split('(')[0]
                    put_embedding_in_dict(a_word, sw2e, sw2e_other, embeddings)
                    append_to_dict(a2r,a_word,word)


                elif '-' in word:  # elif because no sw1-sw2(meaning) , although there is one word(mean-ing)
                    firstsubword = word.split('-')[0]
                    append_to_dict(fsw2c, firstsubword, word)
                
                
                put_embedding_in_dict(word, sw2e, sw2e_other, embeddings)
    

    for sw in sw2f.keys():
        all_supported_words.remove(sw)
    print('duplicates',all_supported_words)

read_words_file(os.path.join('txt',ur_words),ur_sw2f,ur_sw2e,ur_sw2e_other,ur_embeddings,ur_a2r,ur_fsw2c,encoding='utf-8')
read_words_file(os.path.join('txt',en_words),en_sw2f,en_sw2e,en_sw2e_other,en_embeddings,en_a2r,en_fsw2c,encoding='utf-8')

for k in ['d-'+chr(i) for i in range(ord('a'),ord('z')+1)]:
    if k in en_sw2e.keys():
        en_sw2e.pop(k)

print(f'{ur_sw2e_other.keys()-ur_sw2e.keys()=    }')
print(f'{sorted(en_sw2e_other.keys()-en_sw2e.keys())=   }')

duplicates []
duplicates []
ur_sw2e_other.keys()-ur_sw2e.keys()=    set()
sorted(en_sw2e_other.keys()-en_sw2e.keys())=   ['db', 'dc', 'df', 'dg', 'dj', 'dk', 'dl', 'dm', 'dp', 'dq', 'ds', 'dv', 'dw', 'dx', 'dz', 'hecame', 'hewent', 'icome', 'wecome', 'youcome']


In [99]:
# print(f'{ur_sw2f=}')
# print(f'{ur_sw2e.keys()=}')
# print(f'{ur_sw2e_d.keys()=}')
# print(f'{ur_sw2e_other.keys()-ur_sw2e.keys()=}')
print(ur_a2r,'\n')
print(ur_fsw2c,'\n')

print('\n\n\n\n')

# print(f'{en_sw2f=}')
# print(f'{en_sw2e.keys()=}')
# print(f'{en_sw2e_d.keys()=}')
# print(f'{sorted(en_sw2e_other.keys()-en_sw2e.keys())=}')
print(en_a2r,'\n')
print(en_fsw2c,'\n')

{'میں': ['میں(i)', 'میں(in)'], 'کافی': ['کافی(coffee)'], 'کیا': ['کیا(what)'], 'ہفتہ': ['ہفتہ(week)', 'ہفتہ(saturday)'], 'ہوا': ['ہوا(air)']} 

{'d': ['d-A', 'd-B', 'd-C', 'd-D', 'd-E', 'd-F', 'd-G', 'd-H', 'd-I', 'd-J', 'd-K', 'd-L', 'd-M', 'd-N', 'd-O', 'd-P', 'd-Q', 'd-R', 'd-S', 'd-T', 'd-U', 'd-V', 'd-W', 'd-X', 'd-Y', 'd-Z'], 'آئس': ['آئس-کریم'], 'آب': ['آب-پاشی'], 'آتش': ['آتش-فشاں'], 'آدھا': ['آدھا-گھنٹہ'], 'آپ': ['آپ-کا-نام-کیا-ہے'], 'اسلام': ['اسلام-آباد'], 'اَن': ['اَن-پڑھ'], 'اپنا': ['اپنا-کام-کرو'], 'ایک': ['ایک-لاکھ', 'ایک-کروڑ', 'ایک-ہزار', 'ایک-سو'], 'بادشاہی': ['بادشاہی-مسجد'], 'بال': ['بال-پوائنٹ'], 'بحری': ['بحری-جہاز'], 'بس': ['بس-سٹاپ', 'بس-اسٹاپ'], 'بنگلہ': ['بنگلہ-دیش'], 'بک': ['بک-شاپ'], 'بکرے': ['بکرے-کا-گوشت'], 'بکری': ['بکری-کا-گوشت'], 'تاریخی': ['تاریخی-مقامات'], 'تم': ['تم-آئو'], 'جنگی': ['جنگی-جہاز'], 'جینز': ['جینز-کی-پینٹ'], 'حضرت': ['حضرت-آدمؑ', 'حضرت-آدم', 'حضرت-ابوبکرؓ', 'حضرت-ابوبکر', 'حضرت-ابوبکر-صدیقؓ', 'حضرت-ابوبکر-صدیق', 'حضرت-حوّاؑ', 'حضرت-حوّا'

In [102]:
'they' in en_sw2e.keys()

True

In [119]:
en_mini_embeddings={}
with open('top_eng.csv') as f:
    rows = f.read().split('\n')

    for row in rows[1:10007]:
        word = row.split(',')[0]
        try:
            en_mini_embeddings[word] = en_embeddings[word]
        except:
            print(word)
print(len(en_mini_embeddings))

ur_mini_embeddings={}
with open('top_urd.csv', encoding='utf-8') as f:
    rows = f.read().split('\n')

    for row in rows[1:]:
        word = row.split(',')[1]
        try:
            ur_mini_embeddings[word] = ur_embeddings[word]
        except:
            print(word)

print(len(ur_mini_embeddings))

starsmerchant
sublimedirectory
viewpicture
acdbentity
knowledgestorm
bufing
10000
کهيل
کيجيئے
فيصد
سياسی
سيریز
سيد
بهارتی
تهيں
لاکه
بغير
ليں
تکنيکی
تازہ ترین
بڑه
اچهی
ہاته
سکيں
دیکه
وغيرہ
مير
کهلاڑی
پروفيسر
چودهری
کهلاڑیوں
حيثيت
دیکها
اچها
باتيں
چيز
یونيورسٹی
رکه
نامہ نگار
کهانے
بينالاقوامی
صحيح
حيدر
اچهے
بيماری
کيسے
تحصيل
حقيقت
فيصل
دهماکہ
چهوڑ
یقين
موسيقی
دوده
کهيلوں
تربيت
نيٹ
ذمہ دار
گهنٹے
آنکهوں
سنگه
آٹه
ميران
امير
نيشنل
نتيجہ
کيمپ
دکهائی
جهوٹ
سمجه
قيمتوں
رکهيں
حيات
قوانين
کيں
مجه
بيرونی
گهروں
ميڈیکل
کمپنياں
کيو
کمپيوٹر
سرمایہ کاری
سليم
ميچوں
لينڈ
ادهر
پيچهے
چهوٹی
ٹهيک
ٹيپ
ترقياتی
زیادہ تر
غيرت
مہيا
کهيلنے
کهانا
پيدائش
ضياء
بده
یقينی
لاکهوں
کيجئے
دیکهتے
بسيں
رکهتی
معاشيات
کين
تکليف
صلاحيت
بهوت
سينئر
پچهلے
فيڈریشن
دهماکے
بيمہ
ہمہ جہت
شير
فلميں
ميٹر
زلزلہ زدگان
صحافيوں
پوچها
تيسری
مين
بيٹه
کهيلے
بيٹنگ
کهڑے
یقينا
ناظمين
دکه
عليحدہ
ليتا
سيمی
شدت پسند
نوعيت
پڑه
بيماریوں
ليڈر
رحيم
قيمتيں
منصوبہ بندی
بڑها
بيٹهے
سہولتيں
پهول
دهماکوں
کينسر
بوجه
پيش نظر
لياقت
ساڑهے
بيشتر
ميسر
سفيد
سيکورٹی
بڑ

In [22]:
# def read_embeddings(path):
#     dic={}
#     with open(path, encoding='utf-8') as f:
#         rows, dim = map(int,f.readline().split(' '))
        
#         for i in range(rows):
#             row = f.readline().split(' ')
#             dic[row[0]] = np.array(row[1:], dtype=np.float32)
            
#     return dic

# def write_embeddings(dic,filename,mode,cols,encoding='utf-8'):
#     with open(filename, mode, encoding=encoding) as f:
#         f.write(str(len(dic))+f' {cols}\n')
        
#         for k,v in dic.items():
#             f.write(str(k).replace(' ','_')+' '+' '.join(map(str,v))+'\n')

import json
def read_dict(path):
    with open(path,encoding='utf-8') as f:
        dic = json.load(f)
    return dic

import json
def write_dict(dic,path):
    with open(path, 'w',encoding='utf-8') as f:
        json.dump(dic, f)

# write_dict(ur_sw2f  ,'ur_supportedWord2file.json')
# write_embeddings(ur_sw2e,'ur_supportedWords_embeddings.txt', 'w',cols='300')
# write_dict(ur_a2r   ,'ur_ambiguous2resolved.json')
# write_dict(ur_fsw2c ,'ur_firstSubword2compound.json')
# write_embeddings(ur_mini_embeddings,'ur_mini_embeddings.txt', 'w',cols='300')

# write_dict(en_sw2f  ,'en_supportedWord2file.json')
# write_embeddings(en_sw2e,'en_supportedWords_embeddings.txt', 'w',cols='100')
# write_dict(en_a2r   ,'en_ambiguous2resolved.json')
# write_dict(en_fsw2c ,'en_firstSubword2compound.json')
# write_embeddings(en_mini_embeddings,'en_mini_embeddings.txt', 'w',cols='100')

In [23]:
esw2f = read_dict('en_supportedWord2file.json')
for ak in read_dict('en_ambiguous2resolved.json').keys():
    if ak in esw2f.keys():
        print(ak)

i


In [ ]:
# ea2r = read_dict('en_ambiguous2resolved.json')
# ea2r['you']#.append('you(آپ)')

# esw2f['you(آپ)']=esw2f['you']
# esw2f.pop('you')
esw2f['you']

In [29]:
cw = 'you-go-there'
n = cw.count('-')+1
ws=['i','will','not,', 'you', 'go','there','to','her']
'-'.join(ws[3:3+n]) == cw

True